In [ ]:
#Домашняя работа 1 (21.04)

In [ ]:
Рассматривается БД кораблей, участвовавших во второй мировой войне. Имеются следующие отношения:

Classes (class, type, country, numGuns, bore, displacement)
Ships (name, class, launched)
Battles (name, date)
Outcomes (ship, battle, result)
Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.

Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах).
В отношении Ships записаны название корабля, имя его класса и год спуска на воду.
В отношение Battles включены название и дата битвы, в которой участвовали корабли.
В отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).
Замечания. 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 
2) Потопленный корабль в последующих битвах участия не принимает.

ships.gif




In [1]:
#установка библиотеки
!pip install psycopg2

In [20]:
#проверка подключения PostgreSQL 
import psycopg2
import pandas as pd

DB_HOST = '52.157.159.24'

#####################
#!введите свои реквизиты!

DB_USER = 'student15'
DB_USER_PASSWORD = 'student15_password'
DB_NAME = 'sql_ex'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)

In [25]:
#Задание 1: Вывести name, class по кораблям, выпущенным после 1920
#task1
request = """
select name,class from ships where launched>1920
"""
pd.read_sql_query(request, conn)

,name,class
0,California,Tennessee
1,Iowa,Iowa
2,Missouri,Iowa
3,Musashi,Yamato
4,New Jersey,Iowa
5,North Carolina,North Carolina
6,Washington,North Carolina
7,Wisconsin,Iowa
8,Yamato,Yamato
9,South Dakota,North Carolina


In [10]:
#Задание 2: Вывести name, class по кораблям, выпущенным после 1920, но не позднее 1942
#task2
request = """
select name,class from ships where launched between 1920 and 1942
"""
pd.read_sql_query(request, conn)

,name,class
0,California,Tennessee
1,Musashi,Yamato
2,North Carolina,North Carolina
3,Tennessee,Tennessee
4,Washington,North Carolina
5,Yamato,Yamato
6,South Dakota,North Carolina


In [ ]:
#Задание 3: Какое количество кораблей в каждом классе. Вывести количество и class

In [11]:
#task3
request = """
select class, count(name) as kol 
from ships group by class
"""
pd.read_sql_query(request, conn)


,class,kol
0,Tennessee,2
1,Revenge,4
2,Yamato,2
3,Kongo,4
4,Renown,3
5,North Carolina,3
6,Iowa,4


In [ ]:
#Задание 4: Для классов кораблей, калибр орудий которых не менее 16, укажите класс и страну. (таблица classes)

In [12]:
#task4
request = """
select class, country from classes where bore>=16
"""
pd.read_sql_query(request, conn)

,class,country
0,Iowa,USA
1,North Carolina,USA
2,Yamato,Japan


In [ ]:
#Задание 5: Укажите корабли, потопленные в сражениях в Северной Атлантике (таблица Outcomes, North Atlantic). Вывод: ship.

In [13]:
#task5
request = """
select ship from Outcomes where battle='North Atlantic' and result='sunk'
"""
pd.read_sql_query(request, conn)

,ship
0,Bismarck
1,Hood


In [ ]:
#Задание 6: Вывести название (ship) последнего потопленного корабля

In [14]:
#task6
request = """
select ship, name, date from Battles b,Outcomes o 
where o.battle=b.name  and result='sunk' and 
date=(select max(date) from Battles b, Outcomes o where o.battle=b.name and result='sunk')
"""
pd.read_sql_query(request, conn)

,ship,name,date
0,Fuso,Surigao Strait,1944-10-25
1,Yamashiro,Surigao Strait,1944-10-25


In [ ]:
#Задание 7: Вывести название корабля (ship) и класс (class) последнего потопленного корабля

In [15]:
#task7
request = """
select o.ship, s.class  from Battles b,Outcomes o, ships s 
where o.battle=b.name  and result='sunk' and o.ship =s.name and
date=(select max(date) from Battles b, Outcomes o where o.battle=b.name and result='sunk')

"""
pd.read_sql_query(request, conn)

,ship,class


In [ ]:
#Задание 8: Вывести все потопленные корабли, у которых калибр орудий не менее 16, и которые потоплены. Вывод: ship, class

In [18]:
#task8
request = """
select o.ship, c.class from Outcomes o
join ships s on (o.ship=s.name)
join classes c on (s.class=c.class)
where o.result='sunk' and c.bore>=16
"""
pd.read_sql_query(request, conn)

,ship,class


In [ ]:
#Задание 9: Вывести все классы кораблей, выпущенные США (таблица classes, country = 'USA'). Вывод: class

In [24]:
#task9
request = """
select class from classes where country = 'USA'
"""
pd.read_sql_query(request, conn)

,class
0,Iowa
1,North Carolina
2,Tennessee


In [ ]:
#Задание 10: Вывести все корабли, выпущенные США (таблица classes & ships, country = 'USA'). Вывод: name, class

In [22]:
#task10
request = """
select name, c.class from classes c, ships s where c.class=s.class and country = 'USA'
"""
pd.read_sql_query(request, conn)

,name,class
0,California,Tennessee
1,Iowa,Iowa
2,Missouri,Iowa
3,New Jersey,Iowa
4,North Carolina,North Carolina
5,Tennessee,Tennessee
6,Washington,North Carolina
7,Wisconsin,Iowa
8,South Dakota,North Carolina
